In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/psychai-convo-gen-data/bengali_qa.xlsx
/kaggle/input/psychai-convo-gen-data/Analyzer Model report and classifications (0-200).xlsx


In [2]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [ ]:
# STEP 1: Load your dataset
df = pd.read_excel("/kaggle/input/psychai-convo-gen-data/Analyzer Model report and classifications.xlsx")  # Replace with your actual file path

df.head(3)

In [ ]:
# STEP 2: Inspect key columns
print(df.columns)

In [ ]:
import pandas as pd
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Assume df is loaded with columns:
# 'Analyzer_Model_Report' (text), 'Classification' (comma-separated labels)

# 1. Clean text
def clean_text(text):
    text = re.sub(r'[\n\r\t]+', ' ', str(text))
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_text'] = df['Analyzer_Model_Report'].apply(clean_text)

# 2. Prepare multi-label targets
df['labels'] = df['Classification'].apply(lambda x: [label.strip() for label in str(x).split(',')])

# 3. MultiLabelBinarizer to encode target labels
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df['labels'])

print("Classes:", mlb.classes_)

# 4. Split data
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_text'], Y, test_size=0.2, random_state=42)

# 5. Vectorize text with TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # adjust max_features as needed
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 6. Train One-vs-Rest Logistic Regression classifier
model = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train_tfidf, y_train)

# 7. Predict & Evaluate
y_pred = model.predict(X_test_tfidf)

print(classification_report(y_test, y_pred, target_names=mlb.classes_))

In [ ]:
# Print formatted labels
for idx, label in enumerate(mlb.classes_, 1):  # start index from 1
    print(f"{idx}: {label}")

In [ ]:
df.head()

In [ ]:
from IPython.display import HTML

# Get the subset of the 'Analyzer_Model_Report' column
subset_df = df['Conversations'][1:3]

# Create HTML with scrollable container
html_content = f"""
<div style="height: 200px; overflow-y: scroll; border: 1px solid #ccc; padding: 10px;">
    {subset_df.to_frame().to_html(index=False, header=True, escape=False)}
</div>
"""

# Display the HTML content
HTML(html_content)

In [ ]:
from IPython.display import HTML

# Get the subset of the 'Analyzer_Model_Report' column
subset_df = df['labels'][1:3]

# Create HTML with scrollable container
html_content = f"""
<div style="height: 200px; overflow-y: scroll; border: 1px solid #ccc; padding: 10px;">
    {subset_df.to_frame().to_html(index=False, header=True, escape=False)}
</div>
"""

# Display the HTML content
HTML(html_content)

In [ ]:
!pip install tensorflow

In [4]:
# Import necessary libraries
import pandas as pd
import numpy as np
import gensim
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import joblib

2025-08-13 23:45:58.489292: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755128758.705942      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755128758.775243      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
# Step 1: Data Preprocessing
# Load your dataset
df = pd.read_excel("/kaggle/input/psychai-convo-gen-data/Analyzer Model report and classifications (0-200).xlsx")  # Replace with your actual file path

df.head()

,Conversations,Analyzer_Model_Report,Classification
0,"PsychAI: হ্যালো, আমি PsychAI। আপনি যা অভিজ্ঞতা...",১. CBT Report:\r\nপ্রাসঙ্গিক ইতিহাস: ক্লায়েন্...,"Depression, Anxiety, Stress"
1,"PsychAI: হ্যালো, আমি PsychAI। আপনি অনেক জটিল স...",১. CBT রিপোর্ট:\r\nপ্রাসঙ্গিক ইতিহাস: ক্লায়েন...,"Anxiety, Stress"
2,"PsychAI: হেলো, আমি PsychAI বলছি। আমি কীভাবে আপ...",১. CBT Report:\r\nপ্রাসঙ্গিক ইতিহাস:\r\nক্লায়...,"Depression, Anxiety, Stress"
3,"PsychAI: হাই, আমি PsychAI। আপনি কীভাবে আছেন আজ...",১. CBT Report:\r\nপ্রাসঙ্গিক ইতিহাস: ক্লায়েন্...,"Anxiety, Stress"
4,"PsychAI: হ্যালো, আমি PsychAI। আজ আপনি কেমন বোধ...",১. CBT রিপোর্ট:\r\nপ্রাসঙ্গিক ইতিহাস: ক্লায়েন...,"Depression, Anxiety, Stress"


In [10]:
# Check the first few rows of the dataset
print("Data Loaded:")
print(df.head())

# Tokenize the text data (Bengali text)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Conversations'])
X = tokenizer.texts_to_sequences(df['Conversations'])
X = pad_sequences(X, padding='post')  # Padding for sequence uniformity

print("Tokenization and Padding Complete.")
print(f"Shape of X after padding: {X.shape}")

Data Loaded:
                                       Conversations  \
0  PsychAI: হ্যালো, আমি PsychAI। আপনি যা অভিজ্ঞতা...   
1  PsychAI: হ্যালো, আমি PsychAI। আপনি অনেক জটিল স...   
2  PsychAI: হেলো, আমি PsychAI বলছি। আমি কীভাবে আপ...   
3  PsychAI: হাই, আমি PsychAI। আপনি কীভাবে আছেন আজ...   
4  PsychAI: হ্যালো, আমি PsychAI। আজ আপনি কেমন বোধ...   

                               Analyzer_Model_Report  \
0  ১. CBT Report:\r\nপ্রাসঙ্গিক ইতিহাস: ক্লায়েন্...   
1  ১. CBT রিপোর্ট:\r\nপ্রাসঙ্গিক ইতিহাস: ক্লায়েন...   
2  ১. CBT Report:\r\nপ্রাসঙ্গিক ইতিহাস:\r\nক্লায়...   
3  ১. CBT Report:\r\nপ্রাসঙ্গিক ইতিহাস: ক্লায়েন্...   
4  ১. CBT রিপোর্ট:\r\nপ্রাসঙ্গিক ইতিহাস: ক্লায়েন...   

                Classification  
0  Depression, Anxiety, Stress  
1              Anxiety, Stress  
2  Depression, Anxiety, Stress  
3              Anxiety, Stress  
4  Depression, Anxiety, Stress  


AttributeError: 'float' object has no attribute 'lower'

In [11]:
# Step 2: Word2Vec Embedding
# Assuming 'tokenized_sentences' is a list of tokenized Bengali sentences (list of list of words)
tokenized_sentences = [sentence.split() for sentence in df['Conversations']]  # Example tokenization
model_w2v = gensim.models.Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1)
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))

# Fill the embedding matrix with Word2Vec vectors
for word, i in tokenizer.word_index.items():
    if word in model_w2v.wv:
        embedding_matrix[i] = model_w2v.wv[word]

print("Word2Vec Embedding Complete.")
print(f"Embedding Matrix Shape: {embedding_matrix.shape}")

AttributeError: 'float' object has no attribute 'split'

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Step 3: Split Data into Train, Validation, and Test
# Extract labels from the 'Classification' column
# The 'Classification' column contains comma-separated class labels for each row.
# We'll split these labels and apply MultiLabelBinarizer for one-hot encoding.

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['Classification'].str.split(', '))  # Split by comma and space, then one-hot encode

# Check the one-hot encoded labels
print("One-hot Encoded Labels:")
print(y[:5])  # Print first 5 rows of one-hot encoded labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data Split into Train, Validation, and Test sets.")
print(f"Shape of X_train: {X_train.shape}, Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}, Shape of y_test: {y_test.shape}")

In [ ]:
from sklearn.utils import class_weight

# Step 4: Handling Imbalanced Data (Class Weights)
# Compute class weights for each class (since we are doing multilabel classification)
class_weights = {}
for i in range(y_train.shape[1]):  # Iterate through each class (column)
    class_weights[i] = class_weight.compute_class_weight('balanced', 
                                                         classes=np.unique(y_train[:, i]), 
                                                         y=y_train[:, i])
    class_weights[i] = dict(enumerate(class_weights[i]))

print("Class Weights Computed.")
print(f"Class Weight Dict for each label: {class_weights}")

In [ ]:
# Step 5: Model Architecture
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=100, 
                    weights=[embedding_matrix], 
                    input_length=X.shape[1], 
                    trainable=False))  # Embedding layer
model.add(LSTM(128, return_sequences=False))  # LSTM layer
model.add(Dropout(0.3))  # Dropout to prevent overfitting
model.add(Dense(y_train.shape[1], activation='sigmoid'))  # Dense layer with sigmoid for multilabel classification

print("Model Architecture Created.")
model.summary()

In [ ]:
# Step 6: Compile Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Model Compiled.")

In [ ]:
# Step 7: Train the Model with EarlyStopping
# We need to modify how we pass class weights to handle multilabel classification correctly.

# For multilabel classification, compute the sample-wise weights for each class
sample_weights = np.ones_like(y_train)  # Start with all sample weights equal to 1

# Compute class weights for each label (for multilabel classification)
for i in range(y_train.shape[1]):  # Iterate through each label (column)
    class_weights_i = class_weight.compute_class_weight('balanced', 
                                                        classes=np.unique(y_train[:, i]), 
                                                        y=y_train[:, i])
    class_weights_dict = dict(enumerate(class_weights_i))
    
    # Assign the computed class weight for each class
    sample_weights[:, i] = [class_weights_dict.get(int(label), 1.0) for label in y_train[:, i]]

# Train the model
model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=64, 
          sample_weight=sample_weights, callbacks=[early_stop])

print("Model Training Complete.")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Step 8: Model Evaluation
# Make predictions using the model
y_pred_prob = model.predict(X_test)

# Convert probabilities to binary predictions (thresholding at 0.5)
y_pred = (y_pred_prob >= 0.5).astype(int)  # Convert predictions to binary (0 or 1)

# Print classification report (multilabel classification)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=mlb.classes_, zero_division=1))

# Plot confusion matrix for each class (multilabel)
# We'll calculate confusion matrices for each individual class and visualize
for i in range(y_train.shape[1]):  # Loop through each class
    cm = confusion_matrix(y_test[:, i], y_pred[:, i])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['0', '1'], yticklabels=['0', '1'])
    plt.title(f"Confusion Matrix for class {mlb.classes_[i]}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

print("Model Evaluation Complete.")

In [ ]:
# Step 9: Save the Model
# Save the model for future use
joblib.dump(model, 'lstm_model.pkl')
print("Model Saved.")

## Automate

In [17]:
# !pip install --upgrade pip setuptools wheel

In [16]:
# !pip install gensim==4.0.1


In [13]:
!pip install autokeras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.7/122.7 kB 5.4 MB/s eta 0:00:00


In [14]:
# Import necessary libraries for AutoKeras and Hyperparameter Tuning (Optuna)
import pandas as pd
import numpy as np
import gensim
import optuna
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MultiLabelBinarizer
import joblib
import autokeras as ak

In [21]:
# Step 1: Data Preprocessing
# Load your dataset
df = pd.read_excel("/kaggle/input/psychai-convo-gen-data/Analyzer Model report and classifications (0-200).xlsx")  # Replace with your actual file path

print(df.info())
df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Conversations          195 non-null    object
 1   Analyzer_Model_Report  195 non-null    object
 2   Classification         200 non-null    object
dtypes: object(3)
memory usage: 4.8+ KB
None


,Conversations,Analyzer_Model_Report,Classification
0,"PsychAI: হ্যালো, আমি PsychAI। আপনি যা অভিজ্ঞতা...",১. CBT Report:\r\nপ্রাসঙ্গিক ইতিহাস: ক্লায়েন্...,"Depression, Anxiety, Stress"
1,"PsychAI: হ্যালো, আমি PsychAI। আপনি অনেক জটিল স...",১. CBT রিপোর্ট:\r\nপ্রাসঙ্গিক ইতিহাস: ক্লায়েন...,"Anxiety, Stress"
2,"PsychAI: হেলো, আমি PsychAI বলছি। আমি কীভাবে আপ...",১. CBT Report:\r\nপ্রাসঙ্গিক ইতিহাস:\r\nক্লায়...,"Depression, Anxiety, Stress"


In [31]:
# Step 1: Data Preprocessing

# Remove rows where 'Conversations' or 'Analyzer_Model_Report' is empty or NaN
df_cleaned = df.dropna(subset=['Conversations', 'Analyzer_Model_Report'])  # Drop rows with NaN in either column

# Remove rows where 'Conversations' or 'Analyzer_Model_Report' is an empty string
df_cleaned = df_cleaned[df_cleaned['Conversations'].str.strip() != '']
df_cleaned = df_cleaned[df_cleaned['Analyzer_Model_Report'].str.strip() != '']

# Remove rows where 'Classification' contains the error message 'Error: can only concatenate str (not "float") to str'
df_cleaned = df_cleaned[df_cleaned['Classification'] != 'Error: can only concatenate str (not "float") to str']

# Display the cleaned DataFrame info
print("Cleaned DataFrame:")
print(df_cleaned.info())

# Show the first few rows of the cleaned DataFrame
df_cleaned.head()


Cleaned DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 195 entries, 0 to 199
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Conversations          195 non-null    object
 1   Analyzer_Model_Report  195 non-null    object
 2   Classification         195 non-null    object
dtypes: object(3)
memory usage: 6.1+ KB
None


,Conversations,Analyzer_Model_Report,Classification
0,"PsychAI: হ্যালো, আমি PsychAI। আপনি যা অভিজ্ঞতা...",১. CBT Report:\r\nপ্রাসঙ্গিক ইতিহাস: ক্লায়েন্...,"Depression, Anxiety, Stress"
1,"PsychAI: হ্যালো, আমি PsychAI। আপনি অনেক জটিল স...",১. CBT রিপোর্ট:\r\nপ্রাসঙ্গিক ইতিহাস: ক্লায়েন...,"Anxiety, Stress"
2,"PsychAI: হেলো, আমি PsychAI বলছি। আমি কীভাবে আপ...",১. CBT Report:\r\nপ্রাসঙ্গিক ইতিহাস:\r\nক্লায়...,"Depression, Anxiety, Stress"
3,"PsychAI: হাই, আমি PsychAI। আপনি কীভাবে আছেন আজ...",১. CBT Report:\r\nপ্রাসঙ্গিক ইতিহাস: ক্লায়েন্...,"Anxiety, Stress"
4,"PsychAI: হ্যালো, আমি PsychAI। আজ আপনি কেমন বোধ...",১. CBT রিপোর্ট:\r\nপ্রাসঙ্গিক ইতিহাস: ক্লায়েন...,"Depression, Anxiety, Stress"


In [32]:
# List of columns to examine
columns_to_check = ['Classification']

# Loop through the columns and get unique values and their counts
for col in columns_to_check:
    if col in df_cleaned.columns:  # Ensure the column exists in the DataFrame
        print(f"Unique values in '{col}' column:")
        print(df_cleaned[col].value_counts(dropna=False))  # dropna=False to include NaN counts
        print("\n" + "-"*50 + "\n")


Unique values in 'Classification' column:
Classification
Anxiety, Stress                                                                                            57
Depression, Anxiety, Stress                                                                                47
Anxiety, Depression, Stress                                                                                12
Anxiety, Stress, Depression                                                                                10
Anxiety                                                                                                     5
Depression, Stress, Anxiety                                                                                 5
Stress, Anxiety                                                                                             4
Depression, Stress                                                                                          4
Depression, Anxiety, Suicidal Tendency                         

In [36]:
df = df_cleaned

In [37]:
# Tokenize the text data (Bengali text)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Conversations'])

print(tokenizer)

# Convert the text data into sequences
X = tokenizer.texts_to_sequences(df['Conversations'])
X = pad_sequences(X, padding='post')  # Padding for sequence uniformity

# Check the shape after padding
print(f"Shape of X after padding: {X.shape}")

Shape of X after padding: (195, 616)


In [38]:
# Apply MultiLabelBinarizer for multilabel classification
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['Classification'].str.split(', '))  # One-hot encode the labels

y

array([[1, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 1, 0]])

In [40]:
# List of symptoms you want to check
target_symptoms = [
    "Depression", "Suicidal Tendency", "Bipolar Disorder", "Stress", "Anxiety", 
    "Post-Traumatic Stress Disorder (PTSD)", "Sleep Awake Disorder", "Binge Eating Disorder", 
    "Dissociative Disorders", "Schizophrenia Spectrum Disorders", "Paranoid Personality Disorder (PPD)"
]

# Get the class labels from the MultiLabelBinarizer
labels = mlb.classes_

# Count the occurrences of each label across all rows
label_counts = np.sum(y, axis=0)

# Create a dictionary of label occurrences
label_occurrences = {labels[i]: label_counts[i] for i in range(len(labels))}

# Check if all target symptoms are present in the labels
missing_symptoms = [symptom for symptom in target_symptoms if symptom not in labels]
if missing_symptoms:
    print("These symptoms are missing in the labels:")
    print(missing_symptoms)
else:
    print("All target symptoms are present in the labels.")

# Print the occurrences for each of the target symptoms
print("\nTarget Symptoms Occurrences:")
for symptom in target_symptoms:
    if symptom in label_occurrences:
        print(f"{symptom}: {label_occurrences[symptom]}")
    else:
        print(f"{symptom}: Not present in labels")


All target symptoms are present in the labels.

Target Symptoms Occurrences:
Depression: 114
Suicidal Tendency: 16
Bipolar Disorder: 3
Stress: 165
Anxiety: 185
Post-Traumatic Stress Disorder (PTSD): 18
Sleep Awake Disorder: 5
Binge Eating Disorder: 5
Dissociative Disorders: 2
Schizophrenia Spectrum Disorders: 1
Paranoid Personality Disorder (PPD): 5


In [41]:
# Split data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Compute class weights for multilabel classification
class_weights = {}
for i in range(y_train.shape[1]):
    class_weights[i] = class_weight.compute_class_weight('balanced', 
                                                         classes=np.unique(y_train[:, i]), 
                                                         y=y_train[:, i])
    class_weights[i] = dict(enumerate(class_weights[i]))

# Step 2: AutoKeras Model Selection (AutoML Integration)
# AutoKeras will automatically search for the best architecture

# Set up AutoKeras Text Model
text_classifier = ak.TextClassifier(max_trials=5,  # Number of models to try
                                   overwrite=True,  # Overwrite existing models
                                   loss='binary_crossentropy',  # Appropriate for multilabel classification
                                   metrics=['accuracy'])

# Train AutoKeras model
text_classifier.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Step 3: Evaluate AutoKeras Model
# Get the best model and evaluate it
best_model = text_classifier.export_model()

# Evaluate the model
y_pred_prob = best_model.predict(X_test)

# Convert probabilities to binary predictions (thresholding at 0.5)
y_pred = (y_pred_prob >= 0.5).astype(int)

# Classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=mlb.classes_, zero_division=1))

# Plot confusion matrix for each class (multilabel)
for i in range(y_train.shape[1]):
    cm = confusion_matrix(y_test[:, i], y_pred[:, i])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['0', '1'], yticklabels=['0', '1'])
    plt.title(f"Confusion Matrix for class {mlb.classes_[i]}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

# Step 4: Save the Best Model
joblib.dump(best_model, 'best_lstm_model.pkl')
print("Best Model Saved.")


2025-08-14 00:14:24.952425: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


ValueError: Expect the data to TextInput to have shape (batch_size, 1), but got input shape [64, 616].

In [ ]:
# Step 1: Data Preprocessing
# Load your dataset
df = pd.read_excel("/kaggle/input/psychai-convo-gen-data/Analyzer Model report and classifications (0-200).xlsx")  # Replace with your actual file path

# Fill missing values if necessary (you can remove this step if you already cleaned it)
df['Conversations'] = df['Conversations'].fillna('')

# Prepare text data (use the raw text for AutoKeras, not tokenized/padded data)
X = df['Conversations'].values  # Use raw text data

# Apply MultiLabelBinarizer for multilabel classification (same as before)
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['Classification'].str.split(', '))  # One-hot encode the labels

# Split data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Compute class weights for multilabel classification
class_weights = {}
for i in range(y_train.shape[1]):
    class_weights[i] = class_weight.compute_class_weight('balanced', 
                                                         classes=np.unique(y_train[:, i]), 
                                                         y=y_train[:, i])
    class_weights[i] = dict(enumerate(class_weights[i]))

# Step 2: AutoKeras Model Selection (AutoML Integration)
# AutoKeras will automatically search for the best architecture

# Set up AutoKeras Text Model
text_classifier = ak.TextClassifier(max_trials=5,  # Number of models to try
                                   overwrite=True,  # Overwrite existing models
                                   loss='binary_crossentropy',  # Appropriate for multilabel classification
                                   metrics=['accuracy'])

# Train AutoKeras model (fit directly on raw text data)
text_classifier.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test))

# Step 3: Evaluate AutoKeras Model
# Get the best model and evaluate it
best_model = text_classifier.export_model()

# Evaluate the model
y_pred_prob = best_model.predict(X_test)

# Convert probabilities to binary predictions (thresholding at 0.5)
y_pred = (y_pred_prob >= 0.5).astype(int)

# Classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=mlb.classes_, zero_division=1))

# Plot confusion matrix for each class (multilabel)
for i in range(y_train.shape[1]):
    cm = confusion_matrix(y_test[:, i], y_pred[:, i])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['0', '1'], yticklabels=['0', '1'])
    plt.title(f"Confusion Matrix for class {mlb.classes_[i]}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

# Step 4: Save the Best Model
joblib.dump(best_model, 'best_lstm_model.pkl')
print("Best Model Saved.")



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
0                 |0                 |classification_head_1/dropout
adam_weight_decay |adam_weight_decay |optimizer
2e-05             |2e-05             |learning_rate
512               |512               |text_block_1/bert_block_1/max_sequence_length

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None,))
  warnings.warn(msg)


https://chatgpt.com/share/689d0c53-e9fc-8007-a2e3-8bb9df3fa3e1 